In [1]:
#dependencies
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Dense, BatchNormalization, Flatten
from keras.callbacks import ModelCheckpoint
import tarfile
import dlib
import time
import os
import re
import math
import itertools as it
from PIL import Image
import csv

Using TensorFlow backend.


In [2]:
#0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral

In [3]:
def optimise_pts(x,y):
    subset=[]
    i=0
    nose_tip=np.asarray((np.mean(x[30:36]),np.mean(y[30:36])))
    leye=np.asarray((np.mean(x[36:42]),np.mean(y[36:42])))
    reye=np.asarray((np.mean(x[42:48]),np.mean(y[42:48])))
    scale_dist=np.mean([np.linalg.norm(nose_tip-leye),np.linalg.norm(nose_tip-reye)])
    for lip_x,lip_y in zip(it.combinations(x[48:68],2),it.combinations(y[48:68],2)):#190
        x0=lip_x[0]-lip_x[1]
        y0=lip_y[0]-lip_y[1]
        dist=np.linalg.norm([x0,y0])
        subset.append(dist)
    for leyebr_x,leyebr_y in zip(x[17:22],y[17:22]):#30
        for leye_x, leye_y in zip(x[36:42],y[36:42]):
            x1= leyebr_x - leye_x
            y1= leyebr_y - leye_y
            dist=np.linalg.norm([x1,y1])
            subset.append(dist)
    for reyebr_x,reyebr_y in zip(x[22:27],y[22:27]):#30
        for reye_x, reye_y in zip(x[42:48],y[42:48]):
            x2= reyebr_x - reye_x
            y2= reyebr_y - reye_y
            dist=np.linalg.norm([x2,y2])
            subset.append(dist)
    for jaw_x, jaw_y in zip(x[5:12],y[5:12]):#7
        x3 = jaw_x-nose_tip[0]
        y3 = jaw_y-nose_tip[1]
        dist = np.linalg.norm([x3,y3])
        subset.append(dist)
    for nl_x,nl_y in zip(x[48:60],y[48:60]):#12==269
        x4 = nl_x-nose_tip[0]
        y4 = nl_y-nose_tip[1]
        dist = np.linalg.norm([x4,y4])
        subset.append(dist)
    subset=np.array(subset)
    subset = subset/scale_dist*100
    return subset

In [4]:
def get_landmarks(frame):
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    detector = dlib.get_frontal_face_detector()
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    clahe_image = clahe.apply(frame)
    detections = detector(clahe_image,1)
    x=[]
    y=[]
    result_data=[]
    for k,d in enumerate(detections): 
        shape = predictor(clahe_image, d)
        for i in range(1,68):
            x.append(float(shape.part(i).x))
            y.append(float(shape.part(i).y))
            cv2.circle(frame, (shape.part(i).x, shape.part(i).y), 1, (0,0,255), thickness=2) 
        result_data.append(optimise_pts(x,y))
    #cv2.imshow("filename", frame)
    return result_data

In [5]:
def classify_jaffe(filename):
    if(re.search(r'(..\.AN)', filename)):
        return 0
    if(re.search(r'(..\.DI)', filename)):
        return 1
    elif(re.search(r'(..\.FE)', filename)):
        return 2
    elif(re.search(r'(..\.HA)', filename)):
        return 3
    elif(re.search(r'(..\.SA)', filename)):
        return 4
    elif(re.search(r'(..\.SU)', filename)):
        return 5
    elif(re.search(r'(..\.NE)', filename)):
        return 6

In [6]:
o_Dir = "data_landmarks"
#dataset_jaffe = "jaffe"
dataset_jaffe = "train_dir"
data_file = []
label=[]
if not os.path.exists(o_Dir):
    print("Making data_landmark directory")
    os.mkdir(o_Dir)
else:
    print("Output directory already exists")
import csv
print("start of pixel appending")
with open(o_Dir+'/'+ dataset_jaffe+" trn_pixels.csv", 'w') as csvfile:
    dw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for (dirpath, dirnames, filenames) in os.walk("jaffe"):
            print(dirpath)
            for filename in filenames:
                if(re.search(r'\.tiff', filename) or re.search(r'\.jpg', filename) or re.search(r'\.png', filename) or re.search(r'\.jpeg', filename)):
                    filename = dirpath+"/"+filename
                    label.append(classify_jaffe(filename))
                    data_file = get_landmarks(cv2.imread(filename,cv2.IMREAD_UNCHANGED))
                    for i in data_file:
                        dw.writerow(i)
print("appended to csv")

Making data_landmark directory
start of pixel appending
jaffe
appended to csv


In [7]:
with open(o_Dir+'/'+ dataset_jaffe+" trn_label.csv", 'w') as csvfile:
    lblw = csv.writer(csvfile)
    lblw.writerow(label)
print("labels appended to csv")

labels appended to csv


In [8]:
### fer database creation
data_comp = tarfile.open("fer2013.tar") 
ds = pd.read_csv(data_comp.extractfile("fer2013/fer2013.csv"))
train = ds[["emotion", "pixels"]][ds["Usage"] == "Training"]
train['pixels'] = train['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
train_pix = np.vstack(train['pixels'].values)
# test = ds[["emotion", "pixels"]][ds["Usage"] == "PublicTest"]
# test['pixels'] = test['pixels'].apply(lambda x: np.fromstring(x, sep=' '))
# test_pix = np.vstack(test['pixels'].values)
train_pix = train_pix.reshape(-1,48,48)
print("all data reshaped")

all data reshaped


In [9]:
train_ind = np.array(train["emotion"])
# test_ind = np.array(test["emotion"])
train_ind.shape#, test_ind.shape

(28709,)

In [10]:
o_Dir = "data_landmarks"
#dataset_fer = "fer2013"
dataset_fer = "train_dir"
temp_image = "saveddd.jpg"
data_file = []
label= train_ind
if not os.path.exists(o_Dir):
    print("Making data_landmark directory")
    os.mkdir(o_Dir)
else:
    print("Output directory already exists")
import csv
print("start of pixel appending")
with open(o_Dir+'/'+ dataset_fer+" trn_pixels.csv", 'a') as csvfile:
    dw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    sh=train_pix.shape[0]
    train_pix = list(train_pix)
    for filen in range(sh):
        filename = Image.fromarray(train_pix[filen]).convert('RGB')
        filename.save(temp_image)
        img = cv2.imread(temp_image,0)
        o_data = get_landmarks(img)
        for i in o_data:
            dw.writerow(i)
print("appended to csv")
with open(o_Dir+'/'+ dataset_fer+" trn_label.csv", 'a') as csvfile:
    lblw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    lblw.writerow(label)
print("labels appended to csv")
#test_pix = test_pix.reshape(-1,48,48,1)

Output directory already exists
start of pixel appending
appended to csv
labels appended to csv


NameError: name 'test_pix' is not defined

In [11]:
##########

In [25]:
# reading csv file as whole
o_Dir = "data_landmarks"
dataset_fer = "train_dir"
train_data=[]
train_label=[]
if not os.path.exists(o_Dir):
    print("no traing file found")
else:
    print("reading")
    import csv
    print("start of pixel reading")
    with open(o_Dir+'/'+ dataset_fer+" trn_pixels.csv", 'r') as csvfile:
        rd = csv.reader(csvfile)
        for row in rd:
            row = list(map(float, row))
            train_data.append(row)
    with open(o_Dir+'/'+ dataset_fer+" trn_label.csv", 'r') as csvfile:
        rd = csv.reader(csvfile)
        for row in rd:
            row = list(map(int, row))
            train_label.append(row)


reading
start of pixel reading


In [26]:
train_data = np.vstack(train_data)
train_data.shape

(17719, 250)

In [24]:
#train_data = np.vstack(train_data)
train_data = train_data.reshape(-1,10,25,1)
train_label = np.array(train_label)  
# test_ind = np.array(test["emotion"])
train_label = np_utils.to_categorical(train_data)
# test_data = test_pix.reshape(-1,48,48,1)
# test_ind = np_utils.to_categorical(test_ind)
train_ind.shape
#     , test_ind.shape

ValueError: cannot reshape array of size 4429750 into shape (48,48,1)

In [ ]:
model = Sequential()
model.add(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal", 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64, 5))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(7))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
import seaborn as sns
plt.figure(figsize=(14,3))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(res.history['loss'], color='b', label='Training Loss')
plt.plot(res.history['val_loss'], color='r', label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(res.history['acc'], color='b', label='Training Accuracy')
plt.plot(res.history['val_acc'], color='r', label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

In [ ]:
score = model.evaluate(test_pix, test_ind, verbose=0)
score

In [ ]:
from keras.models import model_from_json
import numpy as np

class FacialExpressionModel(object):
    EMOTIONS_LIST = ["Angry", "Disgust/fear/surprise","Sad","Happy", "Neutral"]

    def __init__(self, model_json_file, model_weights_file):
        # load model from JSON file
        with open(model_json_file, "r") as json_file:
            loaded_model_json = json_file.read()
            self.loaded_model = model_from_json(loaded_model_json)

        # load weights into the new model
        self.loaded_model.load_weights(model_weights_file)
        print("Model loaded from disk")
        self.loaded_model.summary()

    def predict_emotion(self, img):
        self.preds = self.loaded_model.predict(img)

        return FacialExpressionModel.EMOTIONS_LIST[np.argmax(self.preds)]


if __name__ == '__main__':
    pass

In [ ]:
import cv2
import numpy as np

rgb = cv2.VideoCapture(0)
facec = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
font = cv2.FONT_HERSHEY_SIMPLEX

def __get_data__():
    _, fr = rgb.read()
    gray = cv2.cvtColor(fr, cv2.COLOR_BGR2GRAY)
    faces = facec.detectMultiScale(gray, 1.3, 5)
    return faces, fr, gray

def start_app(cnn):
    skip_frame = 10
    data = []
    flag = False
    ix = 0
    while True:
        ix += 1
        
        faces, fr, gray_fr = __get_data__()
        for (x, y, w, h) in faces:
            fc = gray_fr[y:y+h, x:x+w]
            
            roi = cv2.resize(fc, (48, 48))
            pred = cnn.predict_emotion(roi[np.newaxis, :, :, np.newaxis])

            cv2.putText(fr, pred, (x, y), font, 1, (255, 255, 0), 2)
            cv2.rectangle(fr,(x,y),(x+w,y+h),(255,0,0),2)

        if cv2.waitKey(1) == 27:
            break
        cv2.imshow('Filter', fr)
    cv2.destroyAllWindows()


if __name__ == '__main__':
    model = FacialExpressionModel("model1.json", "chkPt1.h5")
    start_app(model)